***GENERATED CODE FOR airbnbregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_name', 'transformation_label': 'String Indexer'}], 'feature': 'host_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Eric', 'max': 'Rod', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_name'}, {'feature_label': 'host_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'neighbourhood', 'transformation_label': 'String Indexer'}], 'feature': 'neighbourhood', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'District 2', 'max': 'District 2', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'neighbourhood'}, {'feature_label': 'neighbourhood', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('neighbourhood')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'room_type', 'transformation_label': 'String Indexer'}], 'feature': 'room_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Entire home/apt', 'max': 'Entire home/apt', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'room_type'}, {'feature_label': 'room_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('room_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'last_review', 'transformation_label': 'String Indexer'}], 'feature': 'last_review', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10/17/2023', 'max': '9/30/2023', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'last_review'}, {'feature_label': 'last_review', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('last_review')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'date', 'transformation_label': 'String Indexer'}], 'feature': 'date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2012', 'max': '9/8/2015', 'missing': '0', 'distinct': '390'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'date'}, {'feature_label': 'date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('date')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run airbnbregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	dallasairbnb = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/DallasAirbnb.csv', 'filename': 'DallasAirbnb.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(dallasairbnb)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run airbnbregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	airbnbregressionautofe = TransformationMain.run(dallasairbnb,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "47233404.31", "stddev": "56559.25", "min": "47136756", "max": "47318152", "missing": "0"}, "updatedLabel": "id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "host_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4017367.87", "stddev": "1914828.03", "min": "300211", "max": "6063232", "missing": "0"}, "updatedLabel": "host_id"}, {"transformationsData": [{"feature_label": "host_name", "transformation_label": "String Indexer"}], "feature": "host_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Eric", "max": "Rod", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_name"}, {"transformationsData": [{"feature_label": "neighbourhood", "transformation_label": "String Indexer"}], "feature": "neighbourhood", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "District 2", "max": "District 2", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "neighbourhood"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "32.82", "stddev": "0.06", "min": "32.7350709", "max": "32.92887", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "latitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "longitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-96.78", "stddev": "0.03", "min": "-96.82018", "max": "-96.70625", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "longitude"}, {"transformationsData": [{"feature_label": "room_type", "transformation_label": "String Indexer"}], "feature": "room_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Entire home/apt", "max": "Entire home/apt", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "room_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "price", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "280.56", "stddev": "605.37", "min": "29", "max": "4929", "missing": "0"}, "updatedLabel": "price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "minimum_nights", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "11.96", "stddev": "12.13", "min": "2", "max": "30", "missing": "0"}, "updatedLabel": "minimum_nights"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "number_of_reviews", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "44.64", "stddev": "21.43", "min": "1", "max": "73", "missing": "0"}, "updatedLabel": "number_of_reviews"}, {"transformationsData": [{"feature_label": "last_review", "transformation_label": "String Indexer"}], "feature": "last_review", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10/17/2023", "max": "9/30/2023", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "last_review"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviews_per_month", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.48", "stddev": "1.53", "min": "0.08", "max": "5.48", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "reviews_per_month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "calculated_host_listings_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "42.38", "stddev": "36.63", "min": "1", "max": "83", "missing": "0"}, "updatedLabel": "calculated_host_listings_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "availability_365", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "206.53", "stddev": "98.74", "min": "98", "max": "340", "missing": "0"}, "updatedLabel": "availability_365"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "number_of_reviews_ltm", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "40.22", "stddev": "19.21", "min": "0", "max": "63", "missing": "0"}, "updatedLabel": "number_of_reviews_ltm"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "listing_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "29668231.69", "stddev": "41980.59", "min": "29611902", "max": "29807144", "missing": "0"}, "updatedLabel": "listing_id"}, {"transformationsData": [{"feature_label": "date", "transformation_label": "String Indexer"}], "feature": "date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2012", "max": "9/8/2015", "missing": "0", "distinct": "390"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "date"}]}))

	#transformationPostExecutionHook(airbnbregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run airbnbregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(airbnbregressionautofe, ["id", "host_id", "latitude", "longitude", "price", "minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count", "availability_365", "number_of_reviews_ltm", "listing_id", "host_name_stringindexer", "room_type_stringindexer", "last_review_stringindexer", "date_stringindexer"], "neighbourhood_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

